In [1]:
import os
import sys

sys.path.append("..")
os.chdir("/home/sean/HAI/main_server/sumica")

In [2]:
from controllers.tests.learner2 import Learner

In [3]:
learner = Learner("sean", ["webcam0", "webcam1"])

In [4]:
import time
from utils import db
start_time = 1516102234-180
end_time = 1516104406
results = db.labels.find({"username": "sean", "time": {"$gt": start_time, "$lt": end_time}})
label_types = results.distinct("label")
results = list(results)
print(start_time, end_time)

1516102054 1516104406


In [5]:
results

[]

In [6]:
labels = [r["label"] for r in results]
results = [(r["time"], label_types.index(r["label"])) for r in results]
labels = {"activity": results}
labels

{'activity': []}

In [7]:
%load_ext autoreload
%autoreload 2

from flask import Flask

username = "sean"

app = Flask(__name__)
app.config.from_object('config.Config')

with app.app_context():
    from controllers.activitylearner import ActivityLearner
    cams = ActivityLearner.get_cams(username, start_time, end_time)

In [8]:
from controllers.dbreader.imagereader import ImageReader

imreader = ImageReader()

if end_time is None:
    end_time = time.time()

imdata, times = imreader.read_db(username, start_time, end_time, cams, skip_absent=False)

In [10]:
from controllers.tests.learner2 import Learner

learner = Learner(username, cams)
X, meta = learner.create_image_matrix(imdata)

In [11]:
X.shape

(11, 1078)

In [12]:
times

[1516102183.4479513,
 1516102191.210525,
 1516102195.7154136,
 1516102200.3172812,
 1516102205.1525302,
 1516102209.5608864,
 1516102214.0425346,
 1516104147.8266025,
 1516104152.428377,
 1516104156.7281446,
 1516104160.1172004]

In [13]:
intervals = learner.get_down_times(times)
intervals

array([False, False, False, False, False, False,  True, False, False, False], dtype=bool)

In [14]:
import numpy as np

pauses = np.where(intervals)[0]
pauses

array([6])

In [15]:
ups = [0] + (pauses+1).tolist()
downs = (pauses+1).tolist() + [len(times)]

In [16]:
ups, downs

([0, 7], [7, 11])

In [17]:
for start, end in zip(ups, downs):
    print(times[start:end])

[1516102183.4479513, 1516102191.210525, 1516102195.7154136, 1516102200.3172812, 1516102205.1525302, 1516102209.5608864, 1516102214.0425346]
[1516104147.8266025, 1516104152.428377, 1516104156.7281446, 1516104160.1172004]


In [22]:
from controllers.tests.learner2 import Learner

learner = Learner(username, cams)
X, meta = learner.create_image_matrix(imdata)
intervals = learner.calculate_segments(X, times)
intervals

([(0, 7), (7, 11)],
 [(1516102183.4479513, 1516102214.0425346),
  (1516104147.8266025, 1516104160.1172004)])

In [23]:
labels = {'activity': [(1516102215, 0)]}

In [24]:
models, meta = learner.update_models(labels, start_time)

In [25]:
meta.pop("meta")
meta.pop("matrix")
meta.pop("raw_data")
meta

{'segment_times': [(1516102183.4479513, 1516102214.0425346),
  (1516104147.8266025, 1516104160.1172004)],
 'segments': [(0, 7), (7, 11)],
 'times': [1516102183.4479513,
  1516102191.210525,
  1516102195.7154136,
  1516102200.3172812,
  1516102205.1525302,
  1516102209.5608864,
  1516102214.0425346,
  1516104147.8266025,
  1516104152.428377,
  1516104156.7281446,
  1516104160.1172004],
 'train_labels': {'activity': {'augmented': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32),
   'indices': array([], dtype=int64),
   'mapping': [],
   'raw': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32)}}}